# Voice processor PoC

## Goal

Give back to Lucas free time by automatically processing his voice notes and turning them into feedbacks to kart drivers

## Estimated process

Inputs:

- Voice recorded feedbacks splitted by (1) race and (2) turn
- PDF with race results to identify drivers by their kart number

Processing:

- Create the race in the database
- Create the driver/kart record in the database
- Transform "raw" voice feedback into text and store it in the database
- Split the feedbacks by driver and turn

Outputs:

- Generate a nice feedback in text format to be sent via WhatsApp
- Generate a nice audio recorded feedback to be sent via WhatsApp
